In [3]:
import wandb
import pandas as pd
import numpy as np
import texttable as tt
import latextable

In [19]:
EXPERIMENT_GROUP = "exp-9-dropout2d"
AVERAGE_EPOCHS = 5 # or None | must be odd

def get_best_epoch_metrics(run, metric_name, average_epochs=None):
    history = run.history(samples=10000)
    
    # Find the epoch with the best metric value
    best_epoch = history[metric_name].idxmax()
    best_metrics = history.loc[best_epoch]
    
    if average_epochs:
        # Find indices of valid (non-NaN) values
        valid_indices = history[metric_name].dropna().index
        
        # Locate the best epoch index in the valid indices
        best_index_position = valid_indices.get_loc(best_epoch)
        
        # Get the indices of the AVERAGE_EPOCHS epochs to be averaged
        idx_adjustment = (average_epochs - 1) // 2
        start_index = max(0, best_index_position - idx_adjustment)
        end_index = min(len(valid_indices), best_index_position + idx_adjustment + 1)
        selected_indices = valid_indices[start_index:end_index]
        
        # Average the metrics over the selected epochs
        averaged_metrics = history.loc[selected_indices].mean()
        
        print(f"Run {run.url} - Best epoch: {best_epoch} (averaged over {average_epochs}) - {metric_name}: {averaged_metrics[metric_name]}")
        return averaged_metrics, best_epoch
    else:
        print(f"Run {run.url} - Best epoch: {best_epoch} - {metric_name}: {best_metrics[metric_name]}")
        return best_metrics, best_epoch

def gather_metrics_for_experiment_group(experiment_group, average_epochs=False):
    api = wandb.Api()

    # Fetch runs matching the experiment group
    runs = api.runs("crop-classification/messis", filters={"config.experiment_group": experiment_group})
    print(f"Found {len(runs)} runs for experiment group '{experiment_group}'")

    # Organize the runs by setup name
    setups = {}
    for run in runs:
        setup_name = run.config['name']
        if setup_name not in setups:
            setups[setup_name] = {
                'runs': [],
                'metrics': {metric: [] for metric in [
                    'val_f1_tier1_majority',
                    'val_f1_tier2_majority',
                    'val_f1_tier3_majority',
                    'val_weighted_accuracy_tier1_majority',
                    'val_weighted_accuracy_tier2_majority',
                    'val_weighted_accuracy_tier3_majority',
                    'val_precision_tier1_majority',
                    'val_precision_tier2_majority',
                    'val_precision_tier3_majority',
                    'val_recall_tier1_majority',
                    'val_recall_tier2_majority',
                    'val_recall_tier3_majority',
                    'val_cohen_kappa_tier1_majority',
                    'val_cohen_kappa_tier2_majority',
                    'val_cohen_kappa_tier3_majority',
                ]},
                'best_steps': []
            }
        setups[setup_name]['runs'].append(run)

    # Print setup names with number of runs
    for setup_name, setup_data in setups.items():
        print(f"Setup '{setup_name}' - {len(setup_data['runs'])} runs")

    # Gather metrics for each setup
    for setup_name, setup_data in setups.items():
        for run in setup_data['runs']:
            best_metrics, best_epoch = get_best_epoch_metrics(run, 'val_f1_tier3_majority', average_epochs)
            setup_data['best_steps'].append(best_epoch)
            for metric in setup_data['metrics']:
                setup_data['metrics'][metric].append(best_metrics[metric])
    
    # Prepare data for DataFrame
    data = []
    for setup_name, setup_data in setups.items():
        metrics_summary = {
            'setup': setup_name,
            'num_runs': len(setup_data['runs']),
            'average_epochs': average_epochs,
            'run_handles': [run.id for run in setup_data['runs']],
            'best_steps': setup_data['best_steps'],
        }
        for metric, values in setup_data['metrics'].items():
            metrics_summary[f'{metric}_mean'] = np.mean(values)
            metrics_summary[f'{metric}_std'] = np.std(values)
        data.append(metrics_summary)
    
    metrics_df = pd.DataFrame(data)
    
    return metrics_df

# Run the function for a given experiment group
metrics_df = gather_metrics_for_experiment_group(EXPERIMENT_GROUP, average_epochs=AVERAGE_EPOCHS)

# Save the DataFrame to a CSV file
metrics_df.to_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv", index=False)

metrics_df

Found 16 runs for experiment group 'exp-9-dropout2d'
Setup 'p-0.5' - 1 runs
Setup 'p-0.2' - 5 runs
Setup 'refhead-zuericrop' - 5 runs
Setup 'p-0.1' - 5 runs
Run https://wandb.ai/crop-classification/messis/runs/cm0hlw6k - Best epoch: 4858 (averaged over 5) - val_f1_tier3_majority: 0.19199746549129487
Run https://wandb.ai/crop-classification/messis/runs/rouv9lch - Best epoch: 2446 (averaged over 5) - val_f1_tier3_majority: 0.17942703366279603
Run https://wandb.ai/crop-classification/messis/runs/ym9requx - Best epoch: 2697 (averaged over 5) - val_f1_tier3_majority: 0.1837272971868515
Run https://wandb.ai/crop-classification/messis/runs/6hsyjfm2 - Best epoch: 3250 (averaged over 5) - val_f1_tier3_majority: 0.1891927808523178
Run https://wandb.ai/crop-classification/messis/runs/qhqbesf6 - Best epoch: 3300 (averaged over 5) - val_f1_tier3_majority: 0.199286550283432
Run https://wandb.ai/crop-classification/messis/runs/d3zidx38 - Best epoch: 3300 (averaged over 5) - val_f1_tier3_majority: 0.2

,setup,num_runs,average_epochs,run_handles,best_steps,val_f1_tier1_majority_mean,val_f1_tier1_majority_std,val_f1_tier2_majority_mean,val_f1_tier2_majority_std,val_f1_tier3_majority_mean,...,val_recall_tier2_majority_mean,val_recall_tier2_majority_std,val_recall_tier3_majority_mean,val_recall_tier3_majority_std,val_cohen_kappa_tier1_majority_mean,val_cohen_kappa_tier1_majority_std,val_cohen_kappa_tier2_majority_mean,val_cohen_kappa_tier2_majority_std,val_cohen_kappa_tier3_majority_mean,val_cohen_kappa_tier3_majority_std
0,p-0.5,1,5,[cm0hlw6k],[4858],0.363116,0.000000,0.273813,0.000000,0.191997,...,0.266054,0.000000,0.184392,0.000000,0.871411,0.000000,0.747432,0.000000,0.703745,0.000000
1,p-0.2,5,5,"[rouv9lch, ym9requx, 6hsyjfm2, qhqbesf6, d3zid...","[2446, 2697, 3250, 3300, 3300]",0.461478,0.040348,0.304173,0.009767,0.191326,...,0.298009,0.011831,0.184921,0.008820,0.863881,0.007791,0.740391,0.007973,0.687420,0.008463
2,refhead-zuericrop,5,5,"[6vdvcpsa, voz39aqw, tzbb1fpf, yfrzmegx, mhanp...","[2999, 3350, 3099, 2496, 2496]",0.447482,0.050752,0.301528,0.012916,0.193063,...,0.294862,0.014609,0.186544,0.006217,0.861531,0.005354,0.737542,0.007518,0.685056,0.008471
3,p-0.1,5,5,"[lrt47pea, sft8i5l2, hflukrac, il64btbr, 9nyrx...","[3551, 3401, 3953, 2496, 2345]",0.460275,0.043490,0.304586,0.009983,0.196110,...,0.298495,0.010265,0.188820,0.002574,0.864783,0.006117,0.739495,0.007982,0.686381,0.010797


In [22]:
df = pd.read_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv")

df = df.sort_values(by='setup')

# Mapping for short metric names
metric_name_mapping = {
    'val_f1_tier1_majority': 'F1 $T_1$',
    'val_f1_tier2_majority': 'F1 $T_2$',
    'val_f1_tier3_majority': 'F1 $T_3$',
    'val_weighted_accuracy_tier1_majority': 'W. Acc. $T_1$',
    'val_weighted_accuracy_tier2_majority': 'W. Acc. $T_2$',
    'val_weighted_accuracy_tier3_majority': 'W. Acc. $T_3$',
    'val_precision_tier1_majority': 'Precision $T_1$',
    'val_precision_tier2_majority': 'Precision $T_2$',
    'val_precision_tier3_majority': 'Precision $T_3$',
    'val_recall_tier1_majority': 'Recall $T_1$',
    'val_recall_tier2_majority': 'Recall $T_2$',
    'val_recall_tier3_majority': 'Recall $T_3$',
    'val_cohen_kappa_tier1_majority': 'Kappa $T_1$',
    'val_cohen_kappa_tier2_majority': 'Kappa $T_2$',
    'val_cohen_kappa_tier3_majority': 'Kappa $T_3$',
}

# Function to create a LaTeX table with bold best scores
def create_latex_table(df, first_n_metrics=None):
    metrics = list(metric_name_mapping.keys())
    if first_n_metrics:
        metrics = metrics[:first_n_metrics]
    
    # Determine the best score for each metric
    best_scores = {metric: df[f'{metric}_mean'].max() for metric in metrics}
    
    # Initialize the Texttable object
    table = tt.Texttable()
    
    # Define the header and make headers vertical
    headers = ['Metric'] + [f"{setup} ({row['num_runs']}-fold)" for idx, row in df.iterrows() for setup in [row['setup']]]
    # headers = [f'\\rotatebox{{45}}{{{header}}}' for header in headers]

    table.header(headers)
    
    # Set alignment for columns
    cols_align = ["c"] * len(headers)
    cols_align[0] = "l"
    table.set_cols_align(cols_align)
    
    # Populate the table with data
    for metric in metrics:
        row_data = [metric_name_mapping[metric]]
        for idx, row in df.iterrows():
            mean = row[f'{metric}_mean'] * 100
            std = row[f'{metric}_std'] * 100
            value = f'{mean:.1f}\\% ± {std:.1f}\\%'
            if mean == best_scores[metric] * 100:
                value = f'\\textbf{{{value}}}'
            row_data.append(value)
        table.add_row(row_data)
    
    # Generate the LaTeX table
    average_epochs_used = df['average_epochs'].iloc[0]
    caption_averaged =  f"averaged over {average_epochs_used} epochs" if average_epochs_used else ""
    latex_table = latextable.draw_latex(
        table, 
        caption=f"Results for experiment {EXPERIMENT_GROUP}. All metrics calculated on field majority during validation phase {caption_averaged}, with mean and std reported. Best scores shown in bold.", 
        label=f"tab:metrics_comparison_{EXPERIMENT_GROUP}", 
        use_booktabs=True,
        position="h"
    )

    # Add \resizebox to the LaTeX table
    latex_table = latex_table.replace('\\begin{center}', '\\begin{center}\\resizebox{\\textwidth}{!}{')
    latex_table = latex_table.replace('\\end{center}', '}\\end{center}')

    return latex_table

# Generate the LaTeX table
latex_table = create_latex_table(df, first_n_metrics=6)
print(latex_table)

\begin{table}[h]
	\begin{center}\resizebox{\textwidth}{!}{
		\begin{tabular}{lcccc}
			\toprule
			Metric & p-0.1 (5-fold) & p-0.2 (5-fold) & p-0.5 (1-fold) & refhead-zuericrop (5-fold) \\
			\midrule
			F1 $T_1$ & 46.0\% ± 4.3\% & \textbf{46.1\% ± 4.0\%} & 36.3\% ± 0.0\% & 44.7\% ± 5.1\% \\
			F1 $T_2$ & \textbf{30.5\% ± 1.0\%} & 30.4\% ± 1.0\% & 27.4\% ± 0.0\% & 30.2\% ± 1.3\% \\
			F1 $T_3$ & \textbf{19.6\% ± 0.3\%} & 19.1\% ± 1.0\% & 19.2\% ± 0.0\% & 19.3\% ± 0.5\% \\
			W. Acc. $T_1$ & 93.1\% ± 0.3\% & 93.1\% ± 0.4\% & \textbf{93.5\% ± 0.0\%} & 93.0\% ± 0.3\% \\
			W. Acc. $T_2$ & 82.1\% ± 0.6\% & 82.2\% ± 0.6\% & \textbf{82.7\% ± 0.0\%} & 82.0\% ± 0.6\% \\
			W. Acc. $T_3$ & 77.7\% ± 0.8\% & 77.8\% ± 0.6\% & \textbf{78.9\% ± 0.0\%} & 77.6\% ± 0.6\% \\
			\bottomrule
		\end{tabular}
	}\end{center}
	\caption{Results for experiment exp-9-dropout2d. All metrics calculated on field majority during validation phase averaged over 5 epochs, with mean and std reported. Best scores shown i